# L_X_1_2_var : prédiciotn des données physiques target à l'aide d'une ou 2 variables 

#### Librairies 

In [1]:
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import numpy as np


from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_absolute_error, max_error, mean_absolute_percentage_error

import math

from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor

from sklearn.preprocessing import PowerTransformer
import h5py
from keras.models import load_model

import matplotlib.pyplot as plt
import seaborn as sns

#### Données 

In [195]:
df = pd.read_excel('./df_cleaned.xlsx')

In [196]:
# changer les noms de scolonnes avec / 

# Renommer la colonne existante en 'density_bc (g/cm^3)'
df = df.rename(columns={'density_bc (g/cm^3)':'density_bc (g cm^3)'})
df = df.rename(columns={'density_organics (g/cm^3)':'density_organics (g cm^3)'})
df = df.rename(columns={'mr_total/bc':'mr_total bc'})
df = df.rename(columns={'mr_nonBC/BC':'mr_nonBC BC'})


In [197]:
X = df.iloc[:, 2:23]  # données particules
Y = df.iloc[:,23:31]  # données optiques
L = df.iloc[:, [0, 1] + list(range(31, df.shape[1]))]

In [198]:
# Sélection des variables intéressantes 
variables = ['primary_particle_size (nm)', 
             'vol_equi_radius_outer (nm)', 
             'vol_equi_radius_inner (nm)', 
             'equi_mobility_dia (nm)',
                "mass_bc (g)"]

X = X[variables]

**Normalisation des inputs :**

In [201]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

On prépare les échantillons d'entrainement et de test.

In [191]:
X_train, X_test, Y_train, Y_test = train_test_split(
            L, X,
            test_size=0.30,
            random_state=10)

pt = PowerTransformer(method='yeo-johnson')

X_test=X_test.reset_index(drop=True)
Y_test=Y_test.reset_index(drop=True)


X_train_transformed = scaler.fit_transform(X_train)  #pt.fit_transform(X_train)
X_test_transformed =  scaler.fit_transform(X_test)   #pt.transform(X_test)

Y_train_transformed = pd.DataFrame(Y_train, columns=Y_train.columns)
Y_test_transformed = pd.DataFrame(Y_test, columns=Y_test.columns)

In [192]:
print(X_test.columns)
print(Y_test.columns)

Index(['fractal_dimension', 'fraction_of_coating (%)', 'CR', 'BAE'], dtype='object')
Index(['primary_particle_size (nm)', 'vol_equi_radius_outer (nm)',
       'vol_equi_radius_inner (nm)', 'equi_mobility_dia (nm)', 'mass_bc (g)'],
      dtype='object')


## Prédiction de X à l'aide d'une seule variable de L

Dans cette partie on va créer autant de modèle que de variables dans L. Cela va nous permettre de savoir si les prédictions obtenues à partir d'une seule variable sont bonnes ou non. Si les résultats sont satisfaisants cela voudrait dire qu'il n'y aurait pas besoin d'autant de mesure dans L pour prédire les caractéristiques physiques du BC.  

On va entrainer 1 modèle par variable et comparer les prédictions obtenues à l'aide des métriques utilisées habituellement (R2 et MSE). On va également comparer les performances de 3 algorithmes dans cette taches : KRR, ANN et XGBoost. 

### Avec KRR

In [156]:
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
import pandas as pd

# Paramètres pour GridSearchCV
param_grid = {
    'alpha': [0.1, 1, 10],           # Paramètre de régularisation
    'kernel': ['linear', 'rbf'],     # Choix de noyaux
    'gamma': [0.1, 1, 10]            # Paramètre pour le noyau 'rbf'
}



results = []
best_models1 = {}

# Boucle sur chaque colonne de L
for i, col in enumerate(L.columns):
    print(f"Testing column: {col}")
    
    # Extraire la colonne de L voulue pour l'entraînement
    X_train_col = X_train_transformed[:, i].reshape(-1, 1)
    X_test_col = X_test_transformed[:, i].reshape(-1, 1)
    
    # Boucle sur chaque paramètre de X
    for j, target in enumerate(X.columns):
        
        krr = KernelRidge()
        
        # Configurer GridSearchCV
        grid_search = GridSearchCV(
            estimator=krr,
            param_grid=param_grid,
            cv=5,
            scoring='neg_mean_squared_error',
            n_jobs=-1
        )
        
        grid_search.fit(X_train_col, Y_train.iloc[:, j])
        
        # Meilleur modèle
        best_model = grid_search.best_estimator_
        best_models1[(col,target)] = best_model
        
        y_pred = best_model.predict(X_test_col)
        
        # Calcul des métriques
        mse = mean_squared_error(Y_test_transformed.iloc[:, j], y_pred)
        mape = mean_absolute_percentage_error(Y_test_transformed.iloc[:, j], y_pred)
        r2 = r2_score(Y_test_transformed.iloc[:, j], y_pred)
        
        # Stocker les résultats
        results.append({
            'Feature (L)': col,
            'Target (X)': target,
            'MSE': mse,
            'MAPE': mape,
            'R2': r2,
            'Best Params': grid_search.best_params_
        })
        

param_KRR = pd.DataFrame(results)

# Afficher les performances moyennes par parametres Lidar
performance_summary = param_KRR.groupby('Feature (L)')[['MSE', 'MAPE', 'R2']].mean().sort_values(by='MSE')
print(performance_summary)

Testing column: fractal_dimension
Testing column: fraction_of_coating (%)
Testing column: CR
Testing column: BAE
                                  MSE      MAPE        R2
Feature (L)                                              
CR                       11617.445377  1.261599  0.246007
BAE                      11647.922694  1.258296  0.244012
fraction_of_coating (%)  13105.779589  1.607234  0.220304
fractal_dimension        13879.199243  1.659689 -0.002108


**Comparaison des résultats en fonction de la MSE**

In [158]:
from IPython.display import display

# Trouver les meilleurs résultats pour chaque colonne cible (Y)
best_results_by_target = param_KRR.loc[param_KRR.groupby('Target (X)')['MSE'].idxmin()]

# Trier les résultats pour les visualiser clairement
best_results_sortedMSE = best_results_by_target.sort_values(by='MSE')

# Afficher le tableau formaté avec Pandas
print("Meilleurs résultats de MSE par paramètre physique:")
display(best_results_sortedMSE[['Feature (L)', 'Target (X)', 'Best Params', 'MSE']])

Meilleurs résultats de MSE par paramètre physique:


,Feature (L),Target (X),Best Params,MSE
14,CR,mass_bc (g),"{'alpha': 1, 'gamma': 10, 'kernel': 'rbf'}",2.328840e-29
5,fraction_of_coating (%),primary_particle_size (nm),"{'alpha': 0.1, 'gamma': 10, 'kernel': 'rbf'}",3.525752e-01
12,CR,vol_equi_radius_inner (nm),"{'alpha': 1, 'gamma': 10, 'kernel': 'rbf'}",1.032164e+03
11,CR,vol_equi_radius_outer (nm),"{'alpha': 1, 'gamma': 10, 'kernel': 'rbf'}",1.705515e+03
13,CR,equi_mobility_dia (nm),"{'alpha': 1, 'gamma': 10, 'kernel': 'rbf'}",5.534543e+04


**Comparaison des résultats en fonction de R2**

In [159]:
from IPython.display import display

# Trouver les meilleurs résultats pour chaque colonne cible (Y)
best_results_by_target = param_KRR.loc[param_KRR.groupby('Target (X)')['R2'].idxmax()]

# Trier les résultats pour les visualiser clairement
best_results_sortedR2 = best_results_by_target.sort_values(by='R2')

# Afficher le tableau formaté avec Pandas
print("Meilleurs résultats de R2 par paramètre physique:")
display(best_results_sortedR2[['Feature (L)', 'Target (X)', 'Best Params', 'R2']])


Meilleurs résultats de R2 par paramètre physique:


,Feature (L),Target (X),Best Params,R2
14,CR,mass_bc (g),"{'alpha': 1, 'gamma': 10, 'kernel': 'rbf'}",0.148701
13,CR,equi_mobility_dia (nm),"{'alpha': 1, 'gamma': 10, 'kernel': 'rbf'}",0.155031
11,CR,vol_equi_radius_outer (nm),"{'alpha': 1, 'gamma': 10, 'kernel': 'rbf'}",0.182429
12,CR,vol_equi_radius_inner (nm),"{'alpha': 1, 'gamma': 10, 'kernel': 'rbf'}",0.213117
5,fraction_of_coating (%),primary_particle_size (nm),"{'alpha': 0.1, 'gamma': 10, 'kernel': 'rbf'}",0.959844


**Enregistrement du meilleur modèle pour chaque variable prédite pour comparer les prédictions entre les 3 algorithmes**

In [170]:
# Créer un dictionnaire pour stocker les meilleurs modèles par Target
best_models_KRR_dict = {}

# Parcourir chaque ligne de best_results_sorted
for index, row in best_results_sortedMSE.iterrows():
    target = row['Target (X)']
    feature = row['Feature (L)']
    
    # Enregistrer le modèle (ici 'Feature (L)') pour chaque 'Target (X)'
    best_models_KRR_dict[target] = feature

# Afficher le dictionnaire
print(best_models_KRR_dict)

{'mass_bc (g)': 'CR', 'primary_particle_size (nm)': 'fraction_of_coating (%)', 'vol_equi_radius_inner (nm)': 'CR', 'vol_equi_radius_outer (nm)': 'CR', 'equi_mobility_dia (nm)': 'CR'}


In [175]:
import os
import joblib

# Spécifier le dossier de destination pour les modèles
dossier_de_destination = '../Best_model/L_X_1_var/KRR'

# Créer le dossier s'il n'existe pas déjà
os.makedirs(dossier_de_destination, exist_ok=True)

# Parcourir chaque ligne de best_results_sortedMSE
for target in best_models_KRR_dict :
    features = best_models_KRR_dict[target]
    best_model = best_models1[(features , target)]  # Le modèle pour la feature

    # Créer le chemin complet pour enregistrer le modèle
    chemin_complet = os.path.join(dossier_de_destination, f'{target}_best_modelKRR_avec_{features}.joblib')

    # Enregistrer le modèle dans un fichier avec joblib (pour KRR)
    joblib.dump(best_model, chemin_complet)
    
    print(f"Modèle '{features}' pour la target '{target}' sauvegardé sous le nom '{chemin_complet}'")


Modèle 'CR' pour la target 'mass_bc (g)' sauvegardé sous le nom '../Best_model/L_X_1_var/KRR\mass_bc (g)_best_modelKRR_avec_CR.joblib'
Modèle 'fraction_of_coating (%)' pour la target 'primary_particle_size (nm)' sauvegardé sous le nom '../Best_model/L_X_1_var/KRR\primary_particle_size (nm)_best_modelKRR_avec_fraction_of_coating (%).joblib'
Modèle 'CR' pour la target 'vol_equi_radius_inner (nm)' sauvegardé sous le nom '../Best_model/L_X_1_var/KRR\vol_equi_radius_inner (nm)_best_modelKRR_avec_CR.joblib'
Modèle 'CR' pour la target 'vol_equi_radius_outer (nm)' sauvegardé sous le nom '../Best_model/L_X_1_var/KRR\vol_equi_radius_outer (nm)_best_modelKRR_avec_CR.joblib'
Modèle 'CR' pour la target 'equi_mobility_dia (nm)' sauvegardé sous le nom '../Best_model/L_X_1_var/KRR\equi_mobility_dia (nm)_best_modelKRR_avec_CR.joblib'


### Avec GB

In [176]:
import os
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.model_selection import GridSearchCV
import pandas as pd
from joblib import dump  # Importation de joblib pour sauvegarder les modèles

# Paramètres pour GridSearchCV pour GradientBoostingRegressor
param_grid = {
    'n_estimators': [10, 100, 300, 500],
    'learning_rate': [0.05, 0.1, 0.5],
    'max_depth': [2, 3]
}

results = []
best_models2 = {}

# Boucle sur chaque colonne de L (chaque feature)
for i, col in enumerate(L.columns):
    print(f"Testing column: {col}")
    
    # Extraire la colonne de L voulue pour l'entraînement
    X_train_col = X_train_transformed[:, i].reshape(-1, 1)
    X_test_col = X_test_transformed[:, i].reshape(-1, 1)
    
    # Boucle sur chaque paramètre de X (chaque target)
    for j, target in enumerate(X.columns):
        
        gbr = GradientBoostingRegressor()
        
        # Configurer GridSearchCV
        grid_search = GridSearchCV(
            estimator=gbr,
            param_grid=param_grid,
            cv=5,
            scoring='neg_mean_squared_error',  # On peut utiliser 'neg_mean_squared_error' ou 'neg_mean_absolute_percentage_error'
            n_jobs=-1
        )
        
        # Ajuster le modèle avec la feature et la target correspondantes
        grid_search.fit(X_train_col, Y_train.iloc[:, j])
        
        # Meilleur modèle
        best_model = grid_search.best_estimator_
        best_models2[(col,target)] = best_model
        
        y_pred = best_model.predict(X_test_col)
        
        # Calcul des métriques
        mse = mean_squared_error(Y_test.iloc[:, j], y_pred)
        mape = mean_absolute_percentage_error(Y_test.iloc[:, j], y_pred)
        r2 = r2_score(Y_test.iloc[:, j], y_pred)
        
        # Sauvegarder les résultats dans un tableau
        results.append({
            'Feature (L)': col,
            'Target (X)': target,
            'MSE': mse,
            'MAPE': mape,
            'R2': r2,
            'Best Params': grid_search.best_params_
        })


# Créer un DataFrame pour stocker les résultats
param_GB = pd.DataFrame(results)

# Afficher les performances moyennes par features L
performance_summary_GB = param_GB.groupby('Feature (L)')[['MSE', 'MAPE', 'R2']].mean().sort_values(by='MSE')
print(performance_summary_GB)

Testing column: fractal_dimension
Testing column: fraction_of_coating (%)
Testing column: CR
Testing column: BAE
                                  MSE      MAPE        R2
Feature (L)                                              
CR                       11470.438470  1.539671  0.219473
BAE                      11579.694073  1.540051  0.216418
fraction_of_coating (%)  13067.118420  1.610694  0.230714
fractal_dimension        13824.358686  1.665791  0.000413


**Comparaison des résultats en fonction de la MSE**

In [177]:
from IPython.display import display

# Trouver les meilleurs résultats pour chaque colonne cible (Y)
best_results_by_target = param_GB.loc[param_GB.groupby('Target (X)')['MSE'].idxmin()]

# Trier les résultats pour les visualiser clairement
best_results_sortedMSE = best_results_by_target.sort_values(by='MSE')

# Afficher le tableau formaté avec Pandas
print("Meilleurs résultats de MSE par paramètre physique:")
display(best_results_sortedMSE[['Feature (L)', 'Target (X)', 'Best Params', 'MSE']])


Meilleurs résultats de MSE par paramètre physique:


,Feature (L),Target (X),Best Params,MSE
4,fractal_dimension,mass_bc (g),"{'learning_rate': 0.05, 'max_depth': 2, 'n_est...",2.735672e-29
5,fraction_of_coating (%),primary_particle_size (nm),"{'learning_rate': 0.5, 'max_depth': 3, 'n_esti...",2.027253e-16
12,CR,vol_equi_radius_inner (nm),"{'learning_rate': 0.05, 'max_depth': 2, 'n_est...",1.028998e+03
11,CR,vol_equi_radius_outer (nm),"{'learning_rate': 0.05, 'max_depth': 2, 'n_est...",1.692476e+03
13,CR,equi_mobility_dia (nm),"{'learning_rate': 0.05, 'max_depth': 2, 'n_est...",5.462657e+04


Comparaison des résultats en fonction de R2

In [178]:
from IPython.display import display

# Trouver les meilleurs résultats pour chaque colonne cible (Y)
best_results_by_target = param_GB.loc[param_GB.groupby('Target (X)')['R2'].idxmax()]

# Trier les résultats pour les visualiser clairement
best_results_sortedR2 = best_results_by_target.sort_values(by='R2')

# Afficher le tableau formaté avec Pandas
print("Meilleurs résultats de R2 par paramètre physique:")
display(best_results_sortedR2[['Feature (L)', 'Target (X)', 'Best Params',  'R2']])


Meilleurs résultats de R2 par paramètre physique:


,Feature (L),Target (X),Best Params,R2
4,fractal_dimension,mass_bc (g),"{'learning_rate': 0.05, 'max_depth': 2, 'n_est...",-0.000015
13,CR,equi_mobility_dia (nm),"{'learning_rate': 0.05, 'max_depth': 2, 'n_est...",0.166006
11,CR,vol_equi_radius_outer (nm),"{'learning_rate': 0.05, 'max_depth': 2, 'n_est...",0.188680
12,CR,vol_equi_radius_inner (nm),"{'learning_rate': 0.05, 'max_depth': 2, 'n_est...",0.215531
5,fraction_of_coating (%),primary_particle_size (nm),"{'learning_rate': 0.5, 'max_depth': 3, 'n_esti...",1.000000


Enregistrement du meilleur modèle pour chaque variable prédite pour comparer les prédictions entre les 3 algorithmes

In [179]:
# Créer un dictionnaire pour stocker les meilleurs modèles par Target
best_models_GB_dict = {}

# Parcourir chaque ligne de best_results_sorted
for index, row in best_results_sortedMSE.iterrows():
    target = row['Target (X)']
    feature = row['Feature (L)']
    
    # Enregistrer le modèle (ici 'Feature (L)') pour chaque 'Target (X)'
    best_models_GB_dict[target] = feature

# Afficher le dictionnaire
print(best_models_GB_dict)


{'mass_bc (g)': 'fractal_dimension', 'primary_particle_size (nm)': 'fraction_of_coating (%)', 'vol_equi_radius_inner (nm)': 'CR', 'vol_equi_radius_outer (nm)': 'CR', 'equi_mobility_dia (nm)': 'CR'}


In [181]:
dossier_de_destination = '../Best_model/L_X_1_var/GB'

# Créer le dossier s'il n'existe pas déjà
os.makedirs(dossier_de_destination, exist_ok=True)

for target in best_models_KRR_dict :
    features = best_models_KRR_dict[target]
    best_model = best_models1[(features , target)]  # Le modèle pour la feature

    # Créer le chemin complet pour enregistrer le modèle
    chemin_complet = os.path.join(dossier_de_destination, f'{target}_best_modelGB_avec_{features}.joblib')

    # Enregistrer le modèle dans un fichier avec joblib (pour KRR)
    joblib.dump(best_model, chemin_complet)
    
    print(f"Modèle '{features}' pour la target '{target}' sauvegardé sous le nom '{chemin_complet}'")

Modèle 'CR' pour la target 'mass_bc (g)' sauvegardé sous le nom '../Best_model/L_X_1_var/GB\mass_bc (g)_best_modelGB_avec_CR.joblib'
Modèle 'fraction_of_coating (%)' pour la target 'primary_particle_size (nm)' sauvegardé sous le nom '../Best_model/L_X_1_var/GB\primary_particle_size (nm)_best_modelGB_avec_fraction_of_coating (%).joblib'
Modèle 'CR' pour la target 'vol_equi_radius_inner (nm)' sauvegardé sous le nom '../Best_model/L_X_1_var/GB\vol_equi_radius_inner (nm)_best_modelGB_avec_CR.joblib'
Modèle 'CR' pour la target 'vol_equi_radius_outer (nm)' sauvegardé sous le nom '../Best_model/L_X_1_var/GB\vol_equi_radius_outer (nm)_best_modelGB_avec_CR.joblib'
Modèle 'CR' pour la target 'equi_mobility_dia (nm)' sauvegardé sous le nom '../Best_model/L_X_1_var/GB\equi_mobility_dia (nm)_best_modelGB_avec_CR.joblib'


### Prédiction de X à l'aide d'une combinaison de 2 variables de L

Dans cette partie nous allons voir si combiner 2 variables Lidar pour prédire les données physiques peut avoir un impact positif sur la prédiction.  

Nous allons créer des modèles entrainés avec toutes les combinaisons de variables de L possibles (CR + BAE, etc). Nous allons comparer les 3 mêmes types d'algorithmes que dans la partie précédente : KRR, ANN et XGBoost.

 ### Avec KRR

In [193]:
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.model_selection import GridSearchCV
from itertools import combinations
import pandas as pd

# Paramètres pour GridSearchCV
param_grid = {
    'alpha': [0.1, 1, 10],           # Paramètre de régularisation
    'kernel': ['linear', 'rbf'],     # Choix de noyaux
    'gamma': [0.1, 1, 10]            # Paramètre pour le noyau 'rbf'
}

results = []
best_models = {}

# Boucle sur chaque combinaison de deux colonnes de L
for col_pair in combinations(L.columns, 2):
    print(f"Testing columns: {col_pair}")
    
    # Extraire les deux colonnes de L voulues pour l'entraînement
    col_indices = [L.columns.get_loc(col) for col in col_pair]
    X_train_pair = X_train_transformed[:, col_indices]
    X_test_pair = X_test_transformed[:, col_indices]
    
    # Boucle sur chaque cible de X
    for j, target in enumerate(X.columns):
        
        krr = KernelRidge()
        
        # Configurer GridSearchCV
        grid_search = GridSearchCV(
            estimator=krr,
            param_grid=param_grid,
            cv=5,
            scoring='neg_mean_squared_error',
            n_jobs=-1
        )
        
        # Entraîner le modèle sur les deux colonnes sélectionnées de L pour prédire la cible
        grid_search.fit(X_train_pair, Y_train.iloc[:, j])
        
        # Meilleur modèle pour cette combinaison et cible
        best_model = grid_search.best_estimator_
        best_models[(col_pair, target)] = best_model
        
        # Prédire sur l'ensemble de test
        y_pred = best_model.predict(X_test_pair)
        
        # Calcul des métriques
        mse = mean_squared_error(Y_test_transformed.iloc[:, j], y_pred)
        mape = mean_absolute_percentage_error(Y_test_transformed.iloc[:, j], y_pred)
        r2 = r2_score(Y_test_transformed.iloc[:, j], y_pred)
        
        # Stocker les résultats
        results.append({
            'Features (L)': col_pair,
            'Target (X)': target,
            'MSE': mse,
            'MAPE': mape,
            'R2': r2,
            'Best Params': grid_search.best_params_
        })

# Convertir les résultats en DataFrame pour l'analyse
results_df1 = pd.DataFrame(results)


Testing columns: ('fractal_dimension', 'fraction_of_coating (%)')
Testing columns: ('fractal_dimension', 'CR')
Testing columns: ('fractal_dimension', 'BAE')
Testing columns: ('fraction_of_coating (%)', 'CR')
Testing columns: ('fraction_of_coating (%)', 'BAE')
Testing columns: ('CR', 'BAE')


Comparaison des résultats en fonction de la MSE

In [203]:
from IPython.display import display

# Trouver les meilleurs résultats pour chaque colonne cible (X)
best_results_by_target = results_df1.loc[results_df1.groupby('Target (X)')['MSE'].idxmin()]

# Trier les résultats pour les visualiser clairement
best_results_sortedMSE2 = best_results_by_target.sort_values(by='MSE')

# Afficher le tableau formaté avec Pandas
print("Meilleurs résultats de MSE par colonne cible (X) :")
display(best_results_sortedMSE2[['Features (L)', 'Target (X)', 'Best Params', 'MSE']])

Meilleurs résultats de MSE par colonne cible (X) :


,Features (L),Target (X),Best Params,MSE
24,"(fraction_of_coating (%), BAE)",mass_bc (g),"{'alpha': 0.1, 'gamma': 10, 'kernel': 'rbf'}",1.282253e-29
0,"(fractal_dimension, fraction_of_coating (%))",primary_particle_size (nm),"{'alpha': 0.1, 'gamma': 1, 'kernel': 'rbf'}",6.214881e-02
17,"(fraction_of_coating (%), CR)",vol_equi_radius_inner (nm),"{'alpha': 0.1, 'gamma': 10, 'kernel': 'rbf'}",4.268296e+02
16,"(fraction_of_coating (%), CR)",vol_equi_radius_outer (nm),"{'alpha': 0.1, 'gamma': 10, 'kernel': 'rbf'}",6.675058e+02
23,"(fraction_of_coating (%), BAE)",equi_mobility_dia (nm),"{'alpha': 0.1, 'gamma': 10, 'kernel': 'rbf'}",2.393964e+04


Comparaison des résultats en fonction de R2

In [204]:
from IPython.display import display

# Trouver les meilleurs résultats pour chaque colonne cible (X) basés sur le score R2
best_results_by_target = results_df1.loc[results_df1.groupby('Target (X)')['R2'].idxmax()]

# Trier les résultats pour les visualiser clairement
best_results_sortedR22 = best_results_by_target.sort_values(by='R2', ascending=False)

# Afficher le tableau formaté avec Pandas
print("Meilleurs résultats de R2 par colonne cible (X) :")
display(best_results_sortedR22[['Features (L)', 'Target (X)', 'Best Params', 'R2']])

Meilleurs résultats de R2 par colonne cible (X) :


,Features (L),Target (X),Best Params,R2
0,"(fractal_dimension, fraction_of_coating (%))",primary_particle_size (nm),"{'alpha': 0.1, 'gamma': 1, 'kernel': 'rbf'}",0.992922
16,"(fraction_of_coating (%), CR)",vol_equi_radius_outer (nm),"{'alpha': 0.1, 'gamma': 10, 'kernel': 'rbf'}",0.680018
17,"(fraction_of_coating (%), CR)",vol_equi_radius_inner (nm),"{'alpha': 0.1, 'gamma': 10, 'kernel': 'rbf'}",0.674601
23,"(fraction_of_coating (%), BAE)",equi_mobility_dia (nm),"{'alpha': 0.1, 'gamma': 10, 'kernel': 'rbf'}",0.634509
24,"(fraction_of_coating (%), BAE)",mass_bc (g),"{'alpha': 0.1, 'gamma': 10, 'kernel': 'rbf'}",0.531277


Enregistrement du meilleur modèle pour comparer les prédictions en les 3 algorithmes

In [205]:
# Créer un dictionnaire pour stocker les meilleurs modèles par Target
best_models_KRR2var_dict = {}

# Parcourir chaque ligne de best_results_sorted
for index, row in best_results_sortedMSE2.iterrows():
    target = row['Target (X)']
    feature = row['Features (L)']
    
    # Enregistrer le modèle (ici 'Feature (L)') pour chaque 'Target (X)'
    best_models_KRR2var_dict[target] = feature

# Afficher le dictionnaire
print(best_models_KRR2var_dict)

{'mass_bc (g)': ('fraction_of_coating (%)', 'BAE'), 'primary_particle_size (nm)': ('fractal_dimension', 'fraction_of_coating (%)'), 'vol_equi_radius_inner (nm)': ('fraction_of_coating (%)', 'CR'), 'vol_equi_radius_outer (nm)': ('fraction_of_coating (%)', 'CR'), 'equi_mobility_dia (nm)': ('fraction_of_coating (%)', 'BAE')}


In [208]:
dossier_de_destination = '../Best_model/L_X_2_var/'

# Créer le dossier s'il n'existe pas déjà
os.makedirs(dossier_de_destination, exist_ok=True)

for target in best_models_KRR2var_dict :
    features = best_models_KRR2var_dict[target]
    best_model = best_models[(features , target)]  # Le modèle pour la feature

    # Créer le chemin complet pour enregistrer le modèle
    chemin_complet = os.path.join(dossier_de_destination, f'{target}_best_modelKRR_avec_{features}.joblib')

    # Enregistrer le modèle dans un fichier avec joblib (pour KRR)
    joblib.dump(best_model, chemin_complet)
    
    print(f"Modèle '{features}' pour la target '{target}' sauvegardé sous le nom '{chemin_complet}'")

Modèle '('fraction_of_coating (%)', 'BAE')' pour la target 'mass_bc (g)' sauvegardé sous le nom '../Best_model/L_X_2_var/mass_bc (g)_best_modelKRR_avec_('fraction_of_coating (%)', 'BAE').joblib'
Modèle '('fractal_dimension', 'fraction_of_coating (%)')' pour la target 'primary_particle_size (nm)' sauvegardé sous le nom '../Best_model/L_X_2_var/primary_particle_size (nm)_best_modelKRR_avec_('fractal_dimension', 'fraction_of_coating (%)').joblib'
Modèle '('fraction_of_coating (%)', 'CR')' pour la target 'vol_equi_radius_inner (nm)' sauvegardé sous le nom '../Best_model/L_X_2_var/vol_equi_radius_inner (nm)_best_modelKRR_avec_('fraction_of_coating (%)', 'CR').joblib'
Modèle '('fraction_of_coating (%)', 'CR')' pour la target 'vol_equi_radius_outer (nm)' sauvegardé sous le nom '../Best_model/L_X_2_var/vol_equi_radius_outer (nm)_best_modelKRR_avec_('fraction_of_coating (%)', 'CR').joblib'
Modèle '('fraction_of_coating (%)', 'BAE')' pour la target 'equi_mobility_dia (nm)' sauvegardé sous le nom

 ### Avec GB

In [209]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.model_selection import GridSearchCV
from itertools import combinations
import pandas as pd

# Paramètres pour GridSearchCV pour GradientBoostingRegressor
param_grid = {
    'n_estimators': [10, 100, 300, 500],
    'learning_rate': [0.05, 0.1, 0.5],
    'max_depth': [2, 3]
}

results = []
best_models2 = {}

# Boucle sur chaque combinaison de deux colonnes de L
for col_pair in combinations(L.columns, 2):
    print(f"Testing columns: {col_pair}")
    
    # Extraire les deux colonnes de L voulues pour l'entraînement
    col_indices = [L.columns.get_loc(col) for col in col_pair]
    X_train_pair = X_train_transformed[:, col_indices]
    X_test_pair = X_test_transformed[:, col_indices]
    
    # Boucle sur chaque cible de X
    for j, target in enumerate(X.columns):
        
        gbr = GradientBoostingRegressor()
        
        # Configurer GridSearchCV
        grid_search = GridSearchCV(
            estimator=gbr,
            param_grid=param_grid,
            cv=5,
            scoring='neg_mean_squared_error',  # Peut être aussi 'neg_mean_absolute_percentage_error'
            n_jobs=-1
        )
        
        # Entraîner le modèle sur les deux colonnes sélectionnées de L pour prédire la cible
        grid_search.fit(X_train_pair, Y_train.iloc[:, j])
        
        # Meilleur modèle pour cette combinaison et cible
        best_model = grid_search.best_estimator_
        best_models2[(col_pair, target)] = best_model
        
        # Prédire sur l'ensemble de test
        y_pred = best_model.predict(X_test_pair)
        
        # Calcul des métriques
        mse = mean_squared_error(Y_test.iloc[:, j], y_pred)
        mape = mean_absolute_percentage_error(Y_test.iloc[:, j], y_pred)
        r2 = r2_score(Y_test.iloc[:, j], y_pred)
        
        # Stocker les résultats
        results.append({
            'Features (L)': col_pair,
            'Target (X)': target,
            'MSE': mse,
            'MAPE': mape,
            'R2': r2,
            'Best Params': grid_search.best_params_
        })

# Convertir les résultats en DataFrame pour l'analyse
results_df1 = pd.DataFrame(results)

Testing columns: ('fractal_dimension', 'fraction_of_coating (%)')
Testing columns: ('fractal_dimension', 'CR')
Testing columns: ('fractal_dimension', 'BAE')
Testing columns: ('fraction_of_coating (%)', 'CR')
Testing columns: ('fraction_of_coating (%)', 'BAE')
Testing columns: ('CR', 'BAE')


Comparaison des résultats en fonction de la MSE

In [210]:
from IPython.display import display

# Trouver les meilleurs résultats pour chaque colonne cible (X) en minimisant le MSE
best_results_by_target = results_df1.loc[results_df1.groupby('Target (X)')['MSE'].idxmin()]

# Trier les résultats pour les visualiser clairement
best_results_sortedMSE = best_results_by_target.sort_values(by='MSE')

# Afficher le tableau formaté avec Pandas
print("Meilleurs résultats de MSE par paramètre physique (combinaisons de Features (L)):")
display(best_results_sortedMSE[['Features (L)', 'Target (X)', 'Best Params', 'MSE']])


Meilleurs résultats de MSE par paramètre physique (combinaisons de Features (L)):


,Features (L),Target (X),Best Params,MSE
4,"(fractal_dimension, fraction_of_coating (%))",mass_bc (g),"{'learning_rate': 0.05, 'max_depth': 2, 'n_est...",2.735672e-29
0,"(fractal_dimension, fraction_of_coating (%))",primary_particle_size (nm),"{'learning_rate': 0.5, 'max_depth': 3, 'n_esti...",2.027253e-16
17,"(fraction_of_coating (%), CR)",vol_equi_radius_inner (nm),"{'learning_rate': 0.05, 'max_depth': 3, 'n_est...",5.039376e+02
16,"(fraction_of_coating (%), CR)",vol_equi_radius_outer (nm),"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",7.834279e+02
18,"(fraction_of_coating (%), CR)",equi_mobility_dia (nm),"{'learning_rate': 0.05, 'max_depth': 3, 'n_est...",2.819610e+04


Comparaison des résultats en fonction de R2

In [211]:
from IPython.display import display

# Trouver les meilleurs résultats pour chaque colonne cible (X) en maximisant le R2
best_results_by_target = results_df1.loc[results_df1.groupby('Target (X)')['R2'].idxmax()]

# Trier les résultats pour les visualiser clairement
best_results_sortedR2 = best_results_by_target.sort_values(by='R2', ascending=False)

# Afficher le tableau formaté avec Pandas
print("Meilleurs résultats de R2 par paramètre physique (combinaisons de Features (L)):")
display(best_results_sortedR2[['Features (L)', 'Target (X)', 'R2']])


Meilleurs résultats de R2 par paramètre physique (combinaisons de Features (L)):


,Features (L),Target (X),R2
0,"(fractal_dimension, fraction_of_coating (%))",primary_particle_size (nm),1.000000
16,"(fraction_of_coating (%), CR)",vol_equi_radius_outer (nm),0.624449
17,"(fraction_of_coating (%), CR)",vol_equi_radius_inner (nm),0.615817
18,"(fraction_of_coating (%), CR)",equi_mobility_dia (nm),0.569525
4,"(fractal_dimension, fraction_of_coating (%))",mass_bc (g),-0.000015


Enregistrement du meilleur modèle pour comparer les prédictions en les 3 algorithmes

In [212]:
# Créer un dictionnaire pour stocker les meilleurs modèles par Target
best_models_GB2var_dict = {}

# Parcourir chaque ligne de best_results_sorted
for index, row in best_results_sortedMSE.iterrows():
    target = row['Target (X)']
    feature = row['Features (L)']
    
    # Enregistrer le modèle (ici 'Feature (L)') pour chaque 'Target (X)'
    best_models_GB2var_dict[target] = feature

# Afficher le dictionnaire
print(best_models_GB2var_dict)

{'mass_bc (g)': ('fractal_dimension', 'fraction_of_coating (%)'), 'primary_particle_size (nm)': ('fractal_dimension', 'fraction_of_coating (%)'), 'vol_equi_radius_inner (nm)': ('fraction_of_coating (%)', 'CR'), 'vol_equi_radius_outer (nm)': ('fraction_of_coating (%)', 'CR'), 'equi_mobility_dia (nm)': ('fraction_of_coating (%)', 'CR')}


In [213]:
dossier_de_destination = '../Best_model/L_X_2_var/'

# Créer le dossier s'il n'existe pas déjà
os.makedirs(dossier_de_destination, exist_ok=True)

for target in best_models_GB2var_dict :
    features = best_models_GB2var_dict[target]
    best_model = best_models[(features , target)]  # Le modèle pour la feature

    # Créer le chemin complet pour enregistrer le modèle
    chemin_complet = os.path.join(dossier_de_destination, f'{target}_best_modelGB_avec_{features}.joblib')

    # Enregistrer le modèle dans un fichier avec joblib (pour KRR)
    joblib.dump(best_model, chemin_complet)
    
    print(f"Modèle '{features}' pour la target '{target}' sauvegardé sous le nom '{chemin_complet}'")


Modèle '('fractal_dimension', 'fraction_of_coating (%)')' pour la target 'mass_bc (g)' sauvegardé sous le nom '../Best_model/L_X_2_var/mass_bc (g)_best_modelGB_avec_('fractal_dimension', 'fraction_of_coating (%)').joblib'
Modèle '('fractal_dimension', 'fraction_of_coating (%)')' pour la target 'primary_particle_size (nm)' sauvegardé sous le nom '../Best_model/L_X_2_var/primary_particle_size (nm)_best_modelGB_avec_('fractal_dimension', 'fraction_of_coating (%)').joblib'
Modèle '('fraction_of_coating (%)', 'CR')' pour la target 'vol_equi_radius_inner (nm)' sauvegardé sous le nom '../Best_model/L_X_2_var/vol_equi_radius_inner (nm)_best_modelGB_avec_('fraction_of_coating (%)', 'CR').joblib'
Modèle '('fraction_of_coating (%)', 'CR')' pour la target 'vol_equi_radius_outer (nm)' sauvegardé sous le nom '../Best_model/L_X_2_var/vol_equi_radius_outer (nm)_best_modelGB_avec_('fraction_of_coating (%)', 'CR').joblib'
Modèle '('fraction_of_coating (%)', 'CR')' pour la target 'equi_mobility_dia (nm)'